In [ ]:
import pandas as pd
import numpy as np
import pickle

In [12]:
USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS = 950

print("Loading datafile...")
with open('mon_standard.pkl', 'rb') as fi: # mon_standard.pkl in directory
    data = pickle.load(fi)

Timestamps = []
Direction_Only_Sequence = []
Label = []

for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE
    for sample in data[i]:
        size_seq = []
        time_seq = []
        for c in sample:
            dr = 1 if c > 0 else -1
            time_seq.append(abs(c))
            size_seq.append(dr * 512)
        Timestamps.append(time_seq)
        Direction_Only_Sequence.append(size_seq)
        Label.append(label)

data_dict = {
    "Timestamps": Timestamps,
    "Direction_Only_Sequence": Direction_Only_Sequence,
    "Label":Label
}
df = pd.DataFrame(data_dict)

df.head(5)

Loading datafile...


,Timestamps,Direction_Only_Sequence,Label
0,"[0.0, 0.14, 0.14, 0.31, 0.31, 0.51, 0.51, 0.51...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0
1,"[0.0, 0.13, 0.13, 0.31, 0.77, 1.11, 1.11, 1.11...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0
2,"[0.0, 0.11, 0.11, 0.23, 0.97, 1.11, 1.11, 1.11...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0
3,"[0.0, 0.27, 0.27, 0.6, 0.6, 0.88, 0.89, 0.89, ...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0
4,"[0.0, 0.11, 0.11, 0.36, 0.36, 0.6, 0.6, 0.6, 0...","[-512, -512, 512, -512, 512, -512, 512, 512, -...",0


In [13]:
# Drop Timestamps
df = df.drop(columns=["Timestamps"])

# Filter label 0-94
df = df[df['Label'].between(0, 94)]

# Function to adjust the sequence length to a target length
def adjust_sequence_length(sequence, target_length=10000, padding_value=-1):
    if len(sequence) < target_length:
        # If the sequence is shorter than the target length, pad with the specified value
        sequence = sequence + [padding_value] * (target_length - len(sequence))
    else:
        # If the sequence is longer than the target length, truncate it
        sequence = sequence[:target_length]
    return sequence

# Adjust length
df['Direction_Only_Sequence'] = df['Direction_Only_Sequence'].apply(lambda x: adjust_sequence_length(x))

# Convert to NumPy array and add dimension
mon_data = np.array(df['Direction_Only_Sequence'].tolist(), dtype=np.float32)
mon_data = mon_data[:, :, np.newaxis]  # (19000, 10000, 1)

# Save Numpy array as pkl file
with open("mon_data.pkl", "wb") as f:
    pickle.dump(mon_data, f)

In [14]:
print(mon_data.shape, 'mon_data')

(19000, 10000, 1) mon_data
